![alt text](images/HDAT9500Banner.PNG)
<br>

# Chapter 8: Sequential Data, 
# Assessment: Build an HMM from Data (Baum-Welch algorithm)

#####################################################################################

Double-click to write down your name and surname.

**Name:**
Alexander

**Surname:** 
Kruskal

**Honour Pledge** <p>
    
    
Declaration: <p>
    
    
I declare that this assessment item is my own work, except where acknowledged, and has not been submitted for academic credit elsewhere or previously, or produced independently of this course (e.g. for a third party such as your place of employment) and acknowledge that the assessor of this item may, for the purpose of assessing this item: 

    a. Reproduce this assessment item and provide a copy to another member of the University; and/or 
    b. Communicate a copy of this assessment item to a plagiarism checking service (which may then retain a copy of the assessment item on its database for the purpose of future plagiarism checking). 

#####################################################################################

# 1. Introduction

In the previous exercise, we built an HMM from scratch. We could do this because we knew the values of the probabilities for the model in advance. <p>
In this case, we are going to create a model (to fit a model) using some data. But we are going to initialise the probabilies at random. Therefore, the model will be learn from data.<p>
Again, we will use the HMM in an **unsupervised manner**.
    

## 1.1. Aims of the Exercise:
 1. To continue working with HMMs.
 2. To explore sequential data.
 3. To create an HMM using Baum-Welch alrightm.
 4. To use an HMM to work in **unsupervised** manner (**unsupervised learning**). HMM can be used in a supervised way too, but we are not going to study it in this lesson.
 
It aligns with all the learning outcome of our course: 

1.	Distinguish a range of task specific machine learning techniques appropriate for Health Data Science.
2.	Design machine learning tasks for Health Data Science scenarios.
3.	Construct appropriate training and test sets for health research data.


## 1.2. Jupyter Notebook Intructions
1. Read the content of each cell.
2. Where necessary, follow the instructions that are written in each cell.
3. Run/Execute all the cells that contain Python code sequentially (one at a time), using the "Run" button.
4. For those cells in which you are asked to write some code, please write the Python code first and then execute/run the cell.
 
## 1.3. Tips
 1. The square brackets on the left hand side of each cell indicate whether the cell has been executed or not. Empty square brackets mean that the cell has not been excuted, whereas square brackets that contain a number means that the cell has been executed. Run all the cells in sequence, using the "Run" button.
 2. To edit this notebook, just double-click in each cell. In thid document, each cell can be a "Code" cell or "text-Markdown" cell. To choose between these two options, go to the combo-box above. 
 3. If you want to save your notebook, please make sure you press "the floppy disk" icon button above. 
 4. To clean the content of all cells and re-start Notebook, please go to Cell->All Output->Clear

In [1]:
import sys
print(sys.version)
#For this notebook to work, Python must be 3.6.4 or 3.6.5

import numpy as np
import pandas as pd
from IPython.display import display

from plotnine import *

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


# 2. Instructions
1. Let's assume that we do not have the HMM that we created in the previous exercise.
2. In this exercise, we are going to create an HMM from scratch. Let us assume that in this case, we do not know the probabilies/parameters of our HMM. You will initialize all the probabilites at random, except the probability of state "5" that we assume we know it: <p>
    * model.add_transition(s2, s2, 0.00)
    * model.add_transition(s2, s3, 1.00)<p>
3. In contrast, we have some data that we can use to fit/infer our model. Check questions 1g and 1h of the previous exercise (you should have generated at least 20 sequences).
4. We are going to use the function fit() for fit our model.
        https://media.readthedocs.org/pdf/pomegranate/stable/pomegranate.pdf <p>
        See function "fit()"
5. Follow the instructions below. Each of the questions include instructions.
6. Below is a graphical representation of what we know about our HMM. That is, this is the only certain information that we have with respect to our HMM (along with the data sequences that we are using to "train" our model).

![alt text](images/HMM_to_train.PNG)

Figure 1. Initial Probability and transition probabily of state "5" to state I.

<div class="alert alert-block alert-success">**Start Activity 1**</div>

### <font color='blue'> Question 1:  Load the data that you generated in the previous exercise (5 marks)</font> <p>

In [2]:
%store -r sequences
sequence = sequences[:]

In [3]:
print(type(sequence))
print(type(sequence[0]))

<class 'list'>
<class 'numpy.ndarray'>


In [4]:
#!pip install pomegranate
from pomegranate import *
model=HiddenMarkovModel()

In [5]:
#model = HiddenMarkovModel('DNA Decodification')

### <font color='blue'> Question 2: Define the emission probabilities, link the emission probabilities to the states/hidden variables of your HMM and intitialize the transition probabilites. All the probabilities shoud be initialized randomly, except the ones provided above (see Figure). The states will be named "E", "5" and "I" as before. (40 marks). </font> 
<font color='green'>Tip: Read the pomegranate documenation, Hidden Markov Models section: https://media.readthedocs.org/pdf/pomegranate/latest/pomegranate.pdf </font>

In [6]:
def random_DNA_probabilities(seedx):
    '''
    input: seed - random seed
    output: dictionary of descrete probabilities like - {A: .5, C: 0, G: .5, T: 0} summing to 1

    Generates 4 probabilites adding up to 1 by randomly selecting a value between 0 and remaining percentage.
    Shuffles those probabilites so that it is random to which coding the probability is assigned.
    '''

    from random import seed
    from random import randint
    from random import shuffle
    
    seed(seedx) # seed random number generator

    dict_discrete_dist = {} #output dict
    DNA_probabilities =[] #list probabilities
    DNA_codings = ['A', 'C', 'G', 'T'] #list Coding names
    percent_left = 100 #percent left through each iteration
    
    #create 4 probabilites summing to 100
    for i in DNA_codings:
        temp_percent = randint(0, percent_left)
        temp_decimal = temp_percent/100 #make decimal
        DNA_probabilities.append(temp_decimal)
        percent_left = percent_left - temp_percent
    DNA_probabilities[3] = DNA_probabilities[3] + (percent_left/100) #add remaining probability to last one
    
    #round to avoid floating error
    rounder = lambda x: round(x, 2)
    DNA_probabilities = [rounder(i) for i in DNA_probabilities]
    shuffle(DNA_probabilities)

    #make dictionary
    dict_discrete_dist = dict(zip(DNA_codings, DNA_probabilities))
    return(dict_discrete_dist)

In [7]:
def random_transition_probabilities(seedx):
    '''
    input: seed - random seed
    output: list pair of numbers summing to 1
    
    '''

    from random import seed
    from random import randint
    from random import shuffle
    
    seed(seedx) # seed random number generator
    
    #first and remaining percentage
    percent1 = randint(0, 100)
    percent2 = 100 - percent1
    
    #make list and shuffle
    decimal_probs = [percent1, percent2]
    shuffle(decimal_probs)
    
    #devide by 100 and round to avoid floating error
    rounder = lambda x: round(x, 2)
    decimal_probs = [rounder(i/100) for i in decimal_probs]
    
    return(decimal_probs)
    

In [8]:
dE = DiscreteDistribution(random_DNA_probabilities(10))
d5 = DiscreteDistribution(random_DNA_probabilities(27))
dI = DiscreteDistribution(random_DNA_probabilities(1094))

In [9]:
s1 = State(dE, name="E")
s2 = State(d5, name="5")
s3 = State(dI, name="I")

In [10]:
model.add_states(s1, s2, s3)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s1, random_transition_probabilities(1743)[0])
model.add_transition(s1, s2, random_transition_probabilities(1743)[1])
model.add_transition(s2, s2, 0.0)
model.add_transition(s2, s3, 1.0)
model.add_transition(s3, s3, random_transition_probabilities(74)[0])
model.add_transition(s3,model.end, random_transition_probabilities(74)[1])
model.bake()

### <font color='blue'> Question 3: Once you have initilized our HMM with random values, let's fit the model with our data (sequences) in order to "train" the model and find good parameteres. Give an argument of stop_threshold=0.01 (10 marks). </font> <p>
<font color='green'> Tip: Function "fit()" in the pomegranate documenation, Hidden Markov Models section: 
https://media.readthedocs.org/pdf/pomegranate/stable/pomegranate.pdf <p>
         </font>

In [11]:
model.fit(sequence, stop_threshold=0.01)

{
    "class" : "HiddenMarkovModel",
    "name" : "None",
    "start" : {
        "class" : "State",
        "distribution" : null,
        "name" : "None-start",
        "weight" : 1.0
    },
    "end" : {
        "class" : "State",
        "distribution" : null,
        "name" : "None-end",
        "weight" : 1.0
    },
    "states" : [
        {
            "class" : "State",
            "distribution" : {
                "class" : "Distribution",
                "dtype" : "str",
                "name" : "DiscreteDistribution",
                "parameters" : [
                    {
                        "A" : 0.00023678279903128197,
                        "C" : 0.0,
                        "G" : 0.9707011321207472,
                        "T" : 0.029062085080221508
                    }
                ],
                "frozen" : false
            },
            "name" : "5",
            "weight" : 1.0
        },
        {
            "class" : "State",
            "distributio

### <font color='blue'> Question 4: Use the DNA sequence test that we used in the previous exercise and check that you obtain the same result (or very similar) as before. (20 marks) </font> <p>

In [12]:
DNA_test= list('CTTCATGTGAAAGCAGACGTAAGTCA')

In [13]:
print("-".join(state.name for i, state in model.viterbi(DNA_test)[1]))

None-start-E-E-E-E-E-E-E-E-E-E-E-E-E-E-E-E-E-E-5-I-I-I-I-I-I-I-None-end


In [14]:
print('Log probability: {:.2f}'.format(model.log_probability(DNA_test)))
print('This is similar to the model in the exercise, being -40.55 log-probability')

Log probability: -40.23
This is similar to the model in the exercise, being -40.55 log-probability


In [15]:
print('Probability: {}'.format(np.exp(model.log_probability(DNA_test))))
print('This is similar to the model in the exercise, being 2.72*10^-18% probability')

Probability: 3.3808283104923828e-18
This is similar to the model in the exercise, being 2.72*10^-18% probability


### <font color='blue'> Question 5: With all this knowledge, give a definition of what an HMM is, when you can use it and how you can use it. Minimum 200 words-Maximum 600 words approximately (10 marks). </font> <p>

A Hidden Markov Model is a model that uses observable data to predict hidden or unkown information. For example, with the DNA sequencing, we know the actual DNA sequences, but we do not know which of the three states any coding in the sequence is in. With the observed data and knowing that there are 3 states, the Hidden Markove Model can predict the state and transition probablilites to use the known observation of the codings to determine the unkown states of each DNA coding in the sequence.  
In another real world example, you might observe the weather, the time of year, and the day of the week to determine how busy a bakery might be. In that example the known observations are the weather,time of year, and day of the week. With that information, you might be able to predict the hidden information - how many muffins you are likely to sell. That would then empower the business to either bake more or fewer muffins in the morning to avoid waste or potenetial loss from not having enough to sell.

### <font color='blue'> Question 6: Which task or tasks, of those explained in section 4 in the previous exercise, did we carry out here? Give the task(s) and explain how they are used here. (15 marks)</font> <p>

Of the tasks explained in section 4, we did all three tasks.  
Task 1 is learning or training the model. We did this when we fit the model to our randomly initialised model to our list of sequences.  
Task 2 is finding the probability of an observed sequence given the model. We did this by taking the log probability in question 4 to see if it has a simialer log probability to the model that generated the sequences for the test DNA sequence.  
Task 3 is done using the viterbi function printing out the hidden states for the test DNA sequence to make sure it is similar to the expected hidden states for the DNA test sequence. My model has state '5' 8th from the end instead of 7th from the end, but also the probabilites for the DNA encoding (A, C, G, T) for each state has probabilities closely resembling that of the model the sequences were generated from.